In [1]:
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf

# Download MNIST data
mnist = input_data.read_data_sets('/tmp/data/', one_hot=True)
sess = tf.InteractiveSession()

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [12]:
def train_mnist(x, y, y_, iterations=1000, batch_size=100):
    # Define loss and optimizer
    cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))
    train_step = tf.train.AdamOptimizer().minimize(cross_entropy)

    # Train the model
    tf.initialize_all_variables().run()
    for i in range(iterations):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        train_step.run({x: batch_xs, y_: batch_ys})

    # Test the model
    correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    validation_accuracy = accuracy.eval({x: mnist.validation.images, y_: mnist.validation.labels})
    test_accuracy = accuracy.eval({x: mnist.test.images, y_: mnist.test.labels})
    return validation_accuracy, test_accuracy

In [13]:
def deep_mnist(dtype):
    x = tf.placeholder(dtype, [None, 784])
    y_ = tf.placeholder(dtype, [None, 10])
    x_image = tf.reshape(x, [-1,28,28,1])

    stddev = 0.01
    b_init = 0.01

    W_conv1 = tf.Variable(tf.truncated_normal([5, 5, 1, 32], stddev=stddev, dtype=dtype))
    b_conv1 = tf.Variable(tf.constant(b_init, shape=[32], dtype=dtype))
    o_conv1 = tf.nn.conv2d(x_image, W_conv1, strides=[1, 1, 1, 1], padding='SAME')

    h_conv1 = tf.nn.relu(o_conv1 + b_conv1)
    h_pool1 = tf.nn.max_pool(h_conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

    W_conv2 = tf.Variable(tf.truncated_normal([5, 5, 32, 64], stddev=stddev, dtype=dtype))
    b_conv2 = tf.Variable(tf.constant(b_init, shape=[64], dtype=dtype))
    o_conv2 = tf.nn.conv2d(h_pool1, W_conv2, strides=[1, 1, 1, 1], padding='SAME')

    h_conv2 = tf.nn.relu(o_conv2 + b_conv2)
    h_pool2 = tf.nn.max_pool(h_conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

    W_fc1 = tf.Variable(tf.truncated_normal([7 * 7 * 64, 1024], stddev=stddev, dtype=dtype))
    b_fc1 = tf.Variable(tf.constant(b_init, shape=[1024], dtype=dtype))

    h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

    W_fc2 = tf.Variable(tf.truncated_normal([1024, 10], stddev=stddev, dtype=dtype))
    b_fc2 = tf.Variable(tf.constant(b_init, shape=[10], dtype=dtype))

    y = tf.nn.softmax(tf.matmul(h_fc1, W_fc2) + b_fc2)
    return x, y, y_

In [16]:
%%time
# measure training time in float32
x, y, y_ = deep_mnist(tf.float32)
train_mnist(x, y, y_)

CPU times: user 17min 5s, sys: 1min 50s, total: 18min 56s
Wall time: 6min 36s


In [14]:
%%time
# measure training time in float16
x, y, y_ = deep_mnist(tf.float16)
train_mnist(x, y, y_)

CPU times: user 16h 34min 44s, sys: 3min 46s, total: 16h 38min 30s
Wall time: 4h 43min 25s


In [17]:
%%time
# measure training time in float8
x, y, y_ = deep_mnist(tf.float8)
train_mnist(x, y, y_)

AttributeError: module 'tensorflow' has no attribute 'float8'